# Introduction

In this notebook we will explore random and exhaustive search. Specifically we will apply this to a menu optimization problem. Here we assume the menu is 1 dimensional. Our problem is defined as $n$ menu items and each has an importance weight. Our objective is to minimize the weight sum of movement times. 

In [10]:
import matplotlib.pyplot as plt
import random
import numpy as np
import copy
import itertools
import timeit

Here we define our problem. In our case we defined it as a nest list. Each sublist has a name and importance weight. 

In [11]:
#[name, importance]
problem=[
    ["Home", 0.15],
    ["About", 0.25],
    ["Save", 0.15],
    ["Open", 0.1],
    ["Export", 0.4],
    ["Import", 0.6],
    ["Contact", 0.7],
    ["Research", 0.1],
]

One element of the cost term is Fitts law. This is used to calculate the movement time from the start to each slot. Note that we use the height of a slot here in the fitts law formulation instead of width. This is due to how the width is defined in fitts law. See the lecture for more information
Fitts law (Shannon) is defined as $MT=a+b*\log_2(\frac{D}{W}+1)$ We used an arbitrary value for a and b here. The $W$ is
the width of an element along the travel distance.

<img src="figures/fitts.gif">

Since all the items are directly neighboring. We can simply take the index of a slot and multiply that by its height. This would describe a vertical menu.

In [12]:
def fitts_law(slot_index):
    a= 0.03
    b= 0.1
    height = 20 #px
    width =  100 #px
    distance = (slot_index+1/2)*height
    return a+b*np.log2(distance/height + 1)

## Objective
Now it is time to define our objective term. This is the sum over all items in our menu. We weight this by the importance of each item. So a lower movement time is more important for relevant items. 

In [13]:
def objective_linear(menu):
    cost = 0
    for slot_index, item in enumerate(menu):
        cost += item[1]*fitts_law(slot_index)
    return cost

## Random Solver
We define our first solver. This is a random solver. Probably the most basic solver out there. We try random solutions and keep the best. We do this ```n_iterations``` times. 

In [14]:
def random_solver(menu, n_iterations = 50):
    solution = menu
    best_cost = np.inf
    for iteration in range(n_iterations):
        random.shuffle(menu)
        cost = objective_linear(menu)
        if cost<best_cost:
            solution = copy.deepcopy(menu)
            best_cost=cost
    return solution

## Full Solver
Another way to solve this by literally trying every possible combination. We will see that this scales extremely poor. 

In [15]:
def full_solver(menu, n_iterations):
    solution = menu
    best_cost = np.inf
    cost = objective_linear(menu)
    for combination in itertools.permutations(solution, len(solution)):
        cost = objective_linear(combination)
        if cost<best_cost:
                solution = copy.deepcopy(combination)
                best_cost=cost
    return solution

## Running both
Here we run the full solver for different amount of meny items. Notice of for each additional item the solve time increase almost 10 fold.

In [16]:

def run(menu, solver, n_iterations=100):
    solution = solver(menu, n_iterations)
    return solution

print(run(problem, full_solver, None))
%timeit run(problem, full_solver, None)
%timeit run(problem[1:], full_solver, None)
%timeit run(problem[2:], full_solver, None)

(['Contact', 0.7], ['Import', 0.6], ['Export', 0.4], ['About', 0.25], ['Home', 0.15], ['Save', 0.15], ['Open', 0.1], ['Research', 0.1])
684 ms ± 65.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
74.5 ms ± 5.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
10.2 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


We do a similar experiment for the random solver. Here the solve time scales linear with the number of iterations. But the likelyhood of getting a good solution decreases with the number of menu items. 

In [ ]:
print(run(problem, random_solver, n_iterations=1000))
%timeit run(problem, random_solver, n_iterations=10)
%timeit run(problem, random_solver, n_iterations=100)
%timeit run(problem, random_solver, n_iterations=1000)

In [18]:
print(run(problem, random_solver, n_iterations=1000))
%timeit run(problem, random_solver, n_iterations=10)
%timeit run(problem, random_solver, n_iterations=100)
%timeit run(problem, random_solver, n_iterations=1000)

[['Contact', 0.7], ['Import', 0.6], ['Export', 0.4], ['About', 0.25], ['Home', 0.15], ['Save', 0.15], ['Open', 0.1], ['Research', 0.1]]
290 µs ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.48 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
23.6 ms ± 2.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
